En este notebook se cargan los datos tal cual los tenemos provenientes de los expertos.

Tenemos un excell con varios avistamientos, cada uno con:

- Latitud, longitud de la playa.
- Momento de tiempo (Año, mes, día)
- Número de medusas.

Se quieren tener unos avistamientos por cada "recuadro" de la rejilla, igual que se tienen el resto de atributos (temperatura, salinidad etc).

En este notebook se trabaja considerando que los atributos estaban por cada cuarto de grado, pero ahora a lo mejor es diferente.


In [10]:
import pandas as pd

avistamientos_df = pd.read_excel("./Physalia_Ambiental_R.xlsx")
# avistamientos_df = pd.read_excel("../Physala_Data/Datos_Physalia_20171010.xls")


columnas = avistamientos_df.iloc[0]

#Quito las 3 primeras filas, debido al formato de la excell
avistamientos_df = avistamientos_df.iloc[3:] 
avistamientos_df.columns = columnas

# Me quedo solo con los datos de avistamientos
avistamientos_df = avistamientos_df[["Latitud","Longitud","Año","Mes","Día","Avistamientos"]]


avistamientos_df.head()

,Latitud,Longitud,Año,Mes,Día,Avistamientos
3,-25.4,-70.4833,2014,5,1,1
4,-18.4631,-70.3053,2014,5,1,1
5,-27.0658,-70.8259,2014,5,9,13
6,-18.4933,-70.3261,2014,5,10,15
7,-27.122,-70.8576,2014,5,10,23


In [11]:
#Transdormaciones para sacar con fecha (datetime)
avistamientos_fecha_df=avistamientos_df[["Año","Mes","Día"]]
avistamientos_fecha_df.columns = ["year","month","day"]

fecha = pd.to_datetime(avistamientos_fecha_df)

avistamientos_df["Fecha"]=fecha
avistamientos_df=avistamientos_df[["Latitud","Longitud","Fecha","Avistamientos"]]
avistamientos_df.head()

,Latitud,Longitud,Fecha,Avistamientos
3,-25.4,-70.4833,2014-05-01,1
4,-18.4631,-70.3053,2014-05-01,1
5,-27.0658,-70.8259,2014-05-09,13
6,-18.4933,-70.3261,2014-05-10,15
7,-27.122,-70.8576,2014-05-10,23


In [12]:
from math import floor

def floorQuarter(x):
    return (floor(x * 4) / 4.0) 

#Se añaden atributos con la longitud y latitud redondeadas a cada cuarto de grado
avistamientos_df["Lat_floor"] = avistamientos_df.Latitud.map(floorQuarter)
avistamientos_df["Long_floor"] = avistamientos_df.Longitud.map(floorQuarter)

In [13]:
avistamientos_df.head()

,Latitud,Longitud,Fecha,Avistamientos,Lat_floor,Long_floor
3,-25.4,-70.4833,2014-05-01,1,-25.50,-70.5
4,-18.4631,-70.3053,2014-05-01,1,-18.50,-70.5
5,-27.0658,-70.8259,2014-05-09,13,-27.25,-71.0
6,-18.4933,-70.3261,2014-05-10,15,-18.50,-70.5
7,-27.122,-70.8576,2014-05-10,23,-27.25,-71.0


In [14]:
df = avistamientos_df[['Lat_floor', 'Long_floor',"Fecha","Avistamientos"]]
df = df.set_index(['Lat_floor', 'Long_floor',"Fecha"])

#DataFrame total avistamientos de playas en esa cuadricula
df_sum = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).sum()

#DataFrame número de playas en esa cuadricula
df_count = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).count()

#Dataframe con el total de avistamientos y el número de playas
df_join = df_sum.join(df_count,lsuffix="I",rsuffix="R")
df_join.columns=['Suma', 'N_Playas']
df_join['Media_Playas'] = df_join.Suma /df_join.N_Playas

In [15]:
df_join.head()

Suma  N_Playas  Media_Playas
Lat_floor Long_floor Fecha                                   
-42.75    -74.25     2015-05-27     1         1           1.0
-42.00    -74.00     2014-06-04     1         1           1.0
          -73.75     2015-05-23     1         1           1.0
-41.75    -73.75     2015-06-10    20         1          20.0
                     2015-06-11   151         1         151.0

In [16]:
# DataFrame de avistamientos procesado
df_join.to_excel("avistamientos.xlsx")

# Cross correlation

Este es un código que hay que pulir o discutir en una reunión si lo usamos o no.

Se usa la cross correlación para saber el desfase entre 2 series de números artificiales.

https://en.wikipedia.org/wiki/Cross-correlation



In [8]:
import numpy as np
import pandas as pd

# Creación de una serie de datos artificiales

n = 1000
limit_low = 0
limit_high = 0.48
my_data = np.random.normal(0, 0.5, n) \
          + np.abs(np.random.normal(0, 2, n) \
                   * np.sin(np.linspace(0, 3*np.pi, n)) ) \
          + np.sin(np.linspace(0, 5*np.pi, n))**2 \
          + np.sin(np.linspace(1, 6*np.pi, n))**2

scaling = (limit_high - limit_low) / (max(my_data) - min(my_data))
my_data = my_data * scaling
my_data = my_data + (limit_low - min(my_data))

In [9]:
my_data.shape

(1000,)

In [10]:
from pandas import Series

data = Series(my_data)



In [11]:
datas = []
datas_names = []

''' 
Creación de otras series artificiales, 
que están desplazadas respecto la original
'''

for i in range(5):
    datas.append(data.shift(i*5).fillna(0))
    datas_names.append("Serie"+str(i))


In [12]:
df = pd.DataFrame(dict(zip(datas_names,datas)))

In [13]:
#Vemos como Serie0, Serie1 etc son la misma pero estan desplazadas
df.head(21)

,Serie0,Serie1,Serie2,Serie3,Serie4
0,0.122875,0.000000,0.000000,0.000000,0.000000
1,0.110970,0.000000,0.000000,0.000000,0.000000
2,0.118075,0.000000,0.000000,0.000000,0.000000
3,0.040007,0.000000,0.000000,0.000000,0.000000
4,0.161071,0.000000,0.000000,0.000000,0.000000
5,0.164698,0.122875,0.000000,0.000000,0.000000
6,0.071787,0.110970,0.000000,0.000000,0.000000
7,0.071046,0.118075,0.000000,0.000000,0.000000
8,0.148386,0.040007,0.000000,0.000000,0.000000
9,0.067301,0.161071,0.000000,0.000000,0.000000


In [14]:
from numpy.fft import fft, ifft, fft2, ifft2, fftshift
 
def cross_correlation_using_fft(x, y):
    f1 = fft(x)
    f2 = fft(np.flipud(y))
    cc = np.real(ifft(f1 * f2))
    return fftshift(cc)
 
# shift &lt; 0 means that y starts 'shift' time steps before x # shift &gt; 0 means that y starts 'shift' time steps after x
def compute_shift(x, y):
    assert len(x) == len(y)
    c = cross_correlation_using_fft(x, y)
    assert len(c) == len(x)
    zero_index = int(len(x) / 2) - 1
    shift = zero_index - np.argmax(c)
    return shift

In [15]:
# funciona !, encuentra el desfase
compute_shift(df.Serie0,df.Serie3)

15

In [16]:
'''
La idea sería encontrar el "desfase" entre 
los distintos atributos y la clase, en el tiempo

Es decir, a lo mejor no predecimos las medusas a partir
del día anterior, o de hace 2 días, sino lo que nos diga
un método como el anterior.

Pero esto lo dejamos para más adelante.
'''

'\nLa idea sería encontrar el "desfase" entre \nlos distintos atributos y la clase, en el tiempo\n\nEs decir, a lo mejor no predecimos las medusas a partir\ndel día anterior, o de hace 2 días, sino lo que nos diga\nun método como el anterior.\n\nPero esto lo dejamos para más adelante.\n'